# Qiskit Examples
In this notebook we show some examples of the problem solvers provided by Ingenii, based on the [Qiskit Nature package](https://qiskit.org/documentation/nature/).

### Let's get the workspace we'll submit jobs to.

First, you'll need to establish your connection to the workspace using the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/what-is-azure-cli).

If you're not in our JupyterLab space, you'll need to [install the CLI first](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli).

Then, in a terminal, run the `az login` command to take you through the authentication flow. If you don't do this now, you will be prompted later on.

In [ ]:
from ingenii_azure_quantum.utils import get_qiskit_provider

# If you're in our JupyterLab space then the provider details are pre-configured in environment variables.
qiskit_provider = get_qiskit_provider()

# If you're not in our JupyterLab workspace, either:
#  - set the WORKSPACE_SUBSCRIPTION_ID, WORKSPACE_RESOURCE_GROUP, WORKSPACE_LOCATION, and WORKSPACE_NAME environment variables
#  - pass these values directly to the get_workspace function, as below 
# qiskit_provider = get_qiskit_provider(
#     subscription_id="<subscription ID>",
#     resource_group="<resource group name>",
#     location="<location, e.g. EastUS>",
#     name="<workspace name>",
# )

### As a first example, let's look at a protein folding solution
Adapted from the example given by Qiskit Nature: https://qiskit.org/documentation/nature/tutorials/09_Protein_Folding.html

#### Let's formulate the problem details

In [ ]:
from qiskit_nature.problems.sampling.protein_folding import MiyazawaJerniganInteraction
from qiskit.utils import algorithm_globals

# Let's set the random seed 
algorithm_globals.random_seed = 23

# Protein Main Chain
# The Protein consists of a main chain that is a linear chain of aminoacids. For the naming of different residues we use the one-letter code as defined in Ref. [3]. Further details about the naming and the type of aminoacids can also be found in [4].
# For this particular case we demonstrate the generation of the qubit operator in a neuropeptide with the main chain consisting of 7 aminoacids with letter codes APRLRFY (see also [2]).
main_chain = "APRLRFY"

# Side Chains
# Beyond the main chain of the protein there may be aminoacids attached to the residues of the main chain. Our model allows for side chains of the maximum length of one. Elongated side chains would require the introduction of additional penalty terms which are still under development. In this example we do not consider any side chains to keep the real structure of the neuropeptide
# Ingenii note: this is the default of the 'protein_folding_problem' function, so you don't need to pass this
side_chains = [""] * 7

# Interaction between Aminoacids
# For the description of inter-residue contacts for proteins we use knowledge-based (statistical) potentials derived using quasi-chemical approximation. The potentials used here are introduced by Miyazawa, S. and Jernigan, R. L. in [5].
# Beyond this model we also allow for random contact maps (interactions) that provide a random interaction map. One can also introduce a custom interaction map that enhances certain configurations of the protein (e.g. alpha helix, beta sheet etc).
interaction = MiyazawaJerniganInteraction()

# Finally, define the problem using the function provided by the Ingenii package
from ingenii_azure_quantum.qiskit.problems import protein_folding_problem

qubit_operator = protein_folding_problem(
    main_chain, side_chains=side_chains, interaction=interaction,
)
print(qubit_operator)


#### Let's solve this problem using the VQE solver with CVaR expectation values
By default, the problem solves this locally with a simulator, without submitting to the Quantum Workspace

In [ ]:
from ingenii_azure_quantum.qiskit.solvers import vqe_with_cvar

results = vqe_with_cvar(qubit_operator, verbose=True)
for k, v in results.items():
    print(k, v)


#### As a second pass, let's solve this problem by submitting it to the Quantum Workspace
We'll need to pass the `qiskit_provider` object we obtained earlier, and optionally the name of the backend to use

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

results = vqe_with_cvar(
    qubit_operator, optimizer=COBYLA(maxiter=30),
    azure_quantum_provider=qiskit_provider,
    azure_backend_name="ionq.simulator",
    verbose=True,
)
for k, v in results.items():
    print(k, v)


### References

- [1] https://en.wikipedia.org/wiki/Levinthal%27s_paradox

- [2] A.Robert, P.Barkoutsos, S.Woerner and I.Tavernelli, Resource-efficient quantum algorithm for protein folding, NPJ Quantum Information, 2021, https://doi.org/10.1038/s41534-021-00368-4

- [3] IUPAC–IUB Commission on Biochemical Nomenclature (1972). “A one-letter notation for aminoacid sequences”. Pure and Applied Chemistry. 31 (4): 641–645. doi:10.1351/pac197231040639. PMID 5080161.

- [4] https://en.wikipedia.org/wiki/Amino_acid

- [5] S. Miyazawa and R. L.Jernigan, Residue – Residue Potentials with a Favorable Contact Pair Term and an Unfavorable High Packing Density Term for Simulation and Threading, J. Mol. Biol.256, 623–644, 1996, Table 3, https://doi.org/10.1006/jmbi.1996.0114

- [6] P.Barkoutsos, G. Nannichini, A.Robert, I.Tavernelli, S.Woerner, Improving Variational Quantum Optimization using CVaR, Quantum 4, 256, 2020, https://doi.org/10.22331/q-2020-04-20-256

### As a second example, let's look at a molecule excited state solver
Adapted from the example given by Qiskit Nature: https://qiskit.org/documentation/nature/tutorials/04_excited_states_solvers.html

#### Let's formulate the problem details

In [ ]:
from qiskit_nature.drivers import Molecule

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]],
    charge=0, multiplicity=1
)

# Finally, define the problem using the function provided by the Ingenii package
from ingenii_azure_quantum.qiskit.problems import electronic_structure_problem

es_problem = electronic_structure_problem(molecule)


#### Let's solve this problem using the QEOM solver
By default, the problem solves this locally with a simulator, without submitting to the Quantum Workspace

In [ ]:
from ingenii_azure_quantum.qiskit.solvers import qeom

results = qeom(es_problem, verbose=True)

for k, v in results.items():
    print(k, v)

#### As a second pass, let's solve this problem by submitting it to the Quantum Workspace
We'll need to pass the `qiskit_provider` object we obtained earlier, and optionally the name of the backend to use

In [ ]:
results = qeom(es_problem, azure_quantum_provider=qiskit_provider,
               azure_backend_name="ionq.simulator",
               verbose=True)


for k, v in results.items():
    print(k, v)